# Hyper Parameter Tunning

In [1]:
# standerd imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Ridge
from sklearn.svm import LinearSVC
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
import pickle as pc
%matplotlib inline

In [2]:
cfl = RandomForestClassifier()
